# Lab 1B: Data Cleaning

**WHAT** This nonmandatory lab consists of several programming and insight exercises/questions.

**WHY** The exercises are meant to familiarize yourself with the basic concepts of data cleaning.

**HOW** Follow the exercises in this notebook either on your own or with a fellow student. If you want to skip right to questions and exercises, find the $\rightarrow$ symbol. 

$\newcommand{\q}[1]{\rightarrow \textbf{Question #1}}$
$\newcommand{\ex}[1]{\rightarrow \textbf{Exercise #1}}$

In this assignment, we will take a look at the LendingClub data set. This data corresponds to applications for loans issued in 2016. In order to use this data later for classification of loan applicants, we need to first explore and clean this data. 
Work your way through these exercises at your own pace and be sure to ask questions to the TA's when you don't understand something.

In [ ]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 1. Reading the Data

We first start by reading the data from the LendingClub file provided in the assignemnt folder. 

In [ ]:
#reading the raw data
df = pd.read_csv('./LendingClub.csv')
df.head()

We then load the data dictionary. This dictionary provides a brief description of the features in the dataset

$\ex{1.1}$ Load the data dictionary from the csv file provided

In [ ]:
data_dictionary = None
# START ANSWER
data_dictionary = pd.read_csv('./DataDictionary.csv')
# END ANSWER

data_dictionary

## 2. Explore Data

$\ex{2.1}$ How many samples and features does the dataset contain?

*Hint: Take a look at the pandas.DataFrame in the Pandas API reference scroll down to look at the list of attributes and methods, to familiarize yourself*

In [ ]:
# START ANSWER
df.shape
# END ANSWER

$\ex{2.2}$ Print the names of the features and their corresponding data types

In [ ]:
# START ANSWER
df.dtypes
# END ANSWER

## 3. Removing unwanted observations

Lets take a look at **loan_status** feature.

$\ex{3.1}$ Find the possible values that the feature takes and their respective frequencies

*Hint: Pandas objects have a method that can do this for you*

In [ ]:
# START ANSWER
df['loan_status'].value_counts()
# END ANSWER

This information can be visualized in the form of a histogram. Below we use the countplot function from the seaborn library to create this figure.

In [ ]:
plt.figure(figsize = (12,8))
g = sns.countplot(x="loan_status",data=df,
                  palette='hls')
g.set_xticklabels(g.get_xticklabels(),rotation=45)
g.set_title("Loan Status", fontsize=20)
g.set_xlabel("Loan Status", fontsize=15)
g.set_ylabel("Loan Amount", fontsize=20)

This data is going to be used to predict if a loan is going to be repaid or not based on the loan application. Therefore, we want to make sure that the samples that we use are relevant to making this prediction.

The samples with the  **loan_status = "Current"** correspond to loans whiach are still active.
These samples can be removed as they do not help in predicting if a loan has been repaid or not.

$\ex{3.2}$ Remove the samples that have **loan_status = Current** from the dataframe

In [ ]:
# START ANSWER
df = df[df['loan_status'] != 'Current']
# END ANSWER

We want to do an analysis for the Pacific Coast states California, Oregon, and Washington (not DC!)

$\ex{3.3}$ Select the appropriate subset of samples

In [ ]:
# START ANSWER
df = df[(df['addr_state'] == 'CA') | (df['addr_state'] == 'OR' ) | (df['addr_state'] =='WA')]
# END ANSWER

## 4. Handling Duplicate Observations

Next we want to make sure that there are no duplicate samples in the dataset.

$\ex{4.1}$ Find and print the duplicate observations in the dataset

*Hint: Take a look at the pandas library*

In [ ]:
# START ANSWER
df[df.duplicated()]
# END ANSWER

$\ex{4.2}$ Now remove the duplicates from the dataframe

In [ ]:
# START ANSWER
df = df.drop_duplicates()
# END ANSWER

## 5. Detecting and Dealing with Highly Correlated Features

Features that are highly correlated to each other do not provide much additional information to the data but they do add complexity. 

$\ex{5.1}$ Compute the correlation matrix for the numeric features and make a heatmap to visualize this.

*Hint: Take a look at the heatmap function in the seaborn library*

In [ ]:
correlation_matrix = None

# START ANSWER
correlation_matrix = df.corr(numeric_only=True)
# END ANSWER

plt.figure(figsize=(8, 6))

# Plot the correlation heatmap
# START ANSWER
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
# END ANSWER
plt.title('Correlation Heatmap')
plt.show()


$\ex{5.2}$ Can you identify the features that have a high correlation?

To ensure that there is no redunt information we want to keep only one of the features in each pair of highly correlated features. 

$\ex{5.3}$ Add code below to remove one of the features in each pair.

In [ ]:
# START ANSWER
drop_list_correlated = ["installment","fico_range_high"]
df = df.drop(drop_list_correlated, axis=1)
# END ANSWER

Above we dealt with the numeric features which had a high correlation. Let's take a look at the non-numeric features.

In [ ]:
object_columns = df.columns[df.dtypes == 'object']
df[object_columns]

$\ex{5.4}$ Can you identify the feature that can be removed without removing any information from the dataset (there is a feature providing redundant information)? Remove this feature from the dataframe.

In [ ]:
# Remove the redundant non-numeric feature
# START ANSWER
df = df.drop(['grade'], axis =1)
# END ANSWER

## 6. Dealing with missing data

Another major aspect of data cleaning is handling missing values. There are several ways in which this can be done. We will explore a few different scenarios below.

$\ex{6.1}$ Find the number of null values in each feature

In [ ]:
# START ANSWER
null_count = df.isnull().sum()
null_count
# END ANSWER

Features that have a high percentage of NaN values are not reliable for prediction. These features are usually removed from the dataset.

$\ex{6.2}$ Remove the feature that has more than 25% missing values

In [ ]:
# START ANSWER
df = df.drop(['settlement_amount'], axis=1)
# END ANSWER

First we inspect the categorical feature **title**

In [ ]:
df['title'].value_counts()

The most appropriate value to replace the NaNs with for title is Other.

$\ex{6.3}$  Raplace the NaN values corresponding to the feature **title** with "Other"

*Hint: take a look at the fillna() method in pandas*

In [ ]:
# START ANSWER
df['title'].fillna(value="Other", inplace=True)
# END ANSWER

Now lets take a look at the **dti** feature. Since only 1 sample has a NaN value for this feature, we can handle them by removing this sample.

$\ex{6.4}$ Remove the samples with NaN values for **dti**

In [ ]:
# START ANSWER
df.dropna(subset=["dti"], inplace=True)
# END ANSWER
df.shape

The final feature with missing values is **bc_util**. This is a numeric feature that has a significant number of NaN values. In order to handle this, we will replace this value with an appropriate metric.

$\ex{6.5}$ Can you think of some apppropriate value that could be used to replace the missing values?

$\ex{6.6}$ Find the mean, median and standard deviation of the values of this feature

In [ ]:
mean_value = None
median_value = None
standard_deviation = None

# START ANSWER
mean_value = df['bc_util'].mean()
median_value = df['bc_util'].median()
standard_deviation = df['bc_util'].std()
# END ANSWER

print("Median:", median_value)
print("Mean:", mean_value)
print("Standard Deviation:", standard_deviation)

$\ex{6.7}$ Plot the histogram for this feature.

*Hint: You can use the plot.hist() method in pandas*

In [ ]:
# START ANSWER
df['bc_util'].plot.hist(bins=200)
plt.show()
# END ANSWER

$\ex{6.8}$ Replace the missing values with a suitable value. What value did you choose?

In [ ]:
# START ANSWER
df['bc_util'].fillna(value=mean_value, inplace=True)
# END ANSWER

## 7. Removing features with little predictive value

Sometimes, based on the problem setting, some features do not provide helpful information in terms of prediction. In these cases the features are removed in order to simplify the dataset.

$\ex{7.1}$ Can you identify 2 least useful features in this case? 